In [32]:
## 1. 简单堆叠3折CV分类
from sklearn import datasets
from sklearn.model_selection import train_test_split

'''载入数据'''
X,y = datasets.load_iris(return_X_y=True)

'''不采取分层抽样时的数据集分割'''
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.3)

'''打印各个数据集的形状'''
print(X_train.shape,X_test.shape,y_train.shape,y_test.shape)



(105, 4) (45, 4) (105,) (45,)


In [5]:
## 1. 简单堆叠3折CV分类
from sklearn import datasets

iris = datasets.load_iris()
X, y = iris.data[:, 1:3], iris.target
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier
from mlxtend.classifier import StackingCVClassifier

RANDOM_SEED = 42

clf1 = KNeighborsClassifier(n_neighbors=1)
clf2 = RandomForestClassifier(random_state=RANDOM_SEED)
clf3 = GaussianNB()
lr = LogisticRegression()

# Starting from v0.16.0, StackingCVRegressor supports
# `random_state` to get deterministic result.
sclf = StackingCVClassifier(classifiers=[clf1, clf2, clf3],  # 第一层分类器
                            meta_classifier=lr,   # 第二层分类器
                            random_state=RANDOM_SEED)

print('3-fold cross validation:\n')

for clf, label in zip([clf1, clf2, clf3, sclf], ['KNN', 'Random Forest', 'Naive Bayes','StackingClassifier']):
    scores = cross_val_score(clf, X, y, cv=3, scoring='accuracy')
    print("Accuracy: %0.2f (+/- %0.2f) [%s]" % (scores.mean(), scores.std(), label))

3-fold cross validation:

Accuracy: 0.91 (+/- 0.01) [KNN]
Accuracy: 0.95 (+/- 0.01) [Random Forest]
Accuracy: 0.91 (+/- 0.02) [Naive Bayes]
Accuracy: 0.90 (+/- 0.00) [StackingClassifier]


In [37]:
from sklearn.preprocessing import StandardScaler
ss2 = StandardScaler()
ss2.fit(X_train)
train_data_X_sd = ss2.transform(X_train)
test_data_X_sd = ss2.transform(X_test)
X = train_data_X_sd
X_test = test_data_X_sd
y=y_train
print(X.shape,X_test.shape,y.shape)



(105, 4) (45, 4) (105,)


In [20]:
 
'''模型融合中使用到的各个单模型'''
from sklearn.linear_model import LogisticRegression
from sklearn import svm
import xgboost as xgb
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
 
clfs = [LogisticRegression(C=0.1,max_iter=100),
        xgb.XGBClassifier(max_depth=6,n_estimators=100,num_round = 5),
        RandomForestClassifier(n_estimators=100,max_depth=6,oob_score=True),
        GradientBoostingClassifier(learning_rate=0.3,max_depth=6,n_estimators=100)]

In [58]:
# 创建n_folds
from sklearn.model_selection import StratifiedKFold 
import numpy as np
import pandas as pd

# 创建零矩阵
dataset_blend_train = np.zeros((X.shape[0], len(clfs)))
dataset_blend_test = np.zeros((X_test.shape[0], len(clfs)))

kfold = StratifiedKFold(n_splits=3, shuffle=True, random_state=2019)
print(X.shape,X_test.shape,y.shape)
for j, clf in enumerate(clfs):
    dataset_blend_test_j = np.zeros((X_test.shape[0], 3))
    for n_fold,(tr_idx, val_idx) in enumerate(kfold.split(X, y)):
        print(f'the {n_fold} training start ...')
        tr_x, tr_y, val_x, val_y = X[tr_idx], y[tr_idx], X[val_idx], y[val_idx]
        clf.fit(tr_x, tr_y)
        y_submission = clf.predict_proba(val_x)[:,1]
        dataset_blend_train[val_idx, j] = y_submission
        dataset_blend_test_j[:, n_fold] = clf.predict_proba(X_test)[:, 1]
    dataset_blend_test[:, j] = dataset_blend_test_j.mean(1)
#     print(dataset_blend_test[1])

clf2 = LogisticRegression(C=0.1,max_iter=100)
clf2.fit(dataset_blend_train, y)
y_submission = clf2.predict_proba(dataset_blend_test)[:, 1]
print(y_submission)

(105, 4) (45, 4) (105,)
the 0 training start ...
the 1 training start ...
the 2 training start ...
the 0 training start ...
[21:07:50] WARNING: ../src/learner.cc:541: 
Parameters: { num_round } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[21:07:50] WARNING: ../src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
the 1 training start ...
[21:07:50] WARNING: ../src/learner.cc:541: 
Parameters: { num_round } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  ve

/home/qiwei/dev/anaconda3/lib/python3.7/site-packages/xgboost/sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


the 1 training start ...
the 2 training start ...
the 0 training start ...
the 1 training start ...
the 2 training start ...
[0.18208763 0.18208763 0.18208763 0.18208763 0.18208763 0.18208763
 0.1820876  0.18208762 0.18208763 0.18208763 0.18208763 0.18208763
 0.18208763 0.18208761 0.18208763 0.18208763 0.18208763 0.18208763
 0.18208763 0.18208762 0.18208763 0.18208763 0.18208428 0.18208763
 0.18208763 0.18208763 0.18208762 0.18208763 0.18208755 0.18208763
 0.18208762 0.18208762 0.18208754 0.18208763 0.18208763 0.18208763
 0.18208763 0.18208763 0.18208763 0.18208763 0.18208763 0.18208763
 0.18208763 0.18208763 0.18208763]


In [6]:
from sklearn.preprocessing import StandardScaler
ss2 = StandardScaler()
ss2.fit(X_train)
train_data_X_sd = ss2.transform(X_train)
test_data_X_sd = ss2.transform(X_test)
 

 
##划分train数据集,调用代码,把数据集名字转成和代码一样
X = train_data_X_sd
X_predict = test_data_X_sd
y = train_data_Y
 
'''模型融合中使用到的各个单模型'''
from sklearn.linear_model import LogisticRegression
from sklearn import svm
import xgboost as xgb
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
 
clfs = [LogisticRegression(C=0.1,max_iter=100),
        xgb.XGBClassifier(max_depth=6,n_estimators=100,num_round = 5),
        RandomForestClassifier(n_estimators=100,max_depth=6,oob_score=True),
        GradientBoostingClassifier(learning_rate=0.3,max_depth=6,n_estimators=100)]
 
# 创建n_folds
from sklearn.cross_validation import StratifiedKFold
n_folds = 5
skf = list(StratifiedKFold(y, n_folds))
 
# 创建零矩阵
dataset_blend_train = np.zeros((X.shape[0], len(clfs)))
dataset_blend_test = np.zeros((X_predict.shape[0], len(clfs)))
 
# 建立模型
for j, clf in enumerate(clfs):
    '''依次训练各个单模型'''
    # print(j, clf)
    dataset_blend_test_j = np.zeros((X_predict.shape[0], len(skf)))
    for i, (train, test) in enumerate(skf):
        '''使用第i个部分作为预测，剩余的部分来训练模型，获得其预测的输出作为第i部分的新特征。'''
        # print("Fold", i)
        X_train, y_train, X_test, y_test = X[train], y[train], X[test], y[test]
        clf.fit(X_train, y_train)
        y_submission = clf.predict_proba(X_test)[:, 1]
        dataset_blend_train[test, j] = y_submission
        dataset_blend_test_j[:, i] = clf.predict_proba(X_predict)[:, 1]
    '''对于测试集，直接用这k个模型的预测值均值作为新的特征。'''
    dataset_blend_test[:, j] = dataset_blend_test_j.mean(1)
 
# 用建立第二层模型
clf2 = LogisticRegression(C=0.1,max_iter=100)
clf2.fit(dataset_blend_train, y)
y_submission = clf2.predict_proba(dataset_blend_test)[:, 1]
 
test = pd.read_csv("test.csv")
test["Survived"] = clf2.predict(dataset_blend_test)
test[['PassengerId','Survived']].set_index('PassengerId').to_csv('stack3.csv')

NameError: name 'train_data_X' is not defined